### When running this notebook via the Galaxy portal
You can access your data via the dataset number. Using a Python kernel, you can access dataset number 42 with ``handle = open(get(42), 'r')``.
To save data, write your data to a file, and then call ``put('filename.txt')``. The dataset will then be available in your galaxy history.
<br><br>Note that if you are putting/getting to/from a different history than your default history, you must also provide the history-id.
<br><br>More information including available galaxy-related environment variables can be found at https://github.com/bgruening/docker-jupyter-notebook. This notebook is running in a docker container based on the Docker Jupyter container described in that link.


# Quadlepton analysis  (Python) 


This is an example of a simple dilepton analysis, quite similar to the notebook called "Dilepton_analysis_noData.ipynb", but with some differences. The most obvious difference is that we here also include real data. This example also has a slightly more advanced event selection.  

**Notice:** This is *only an example* on how to do this. Feel free to be creative, and to find better and/or more elegant ways of doing the various steps! 

In [1]:
import ROOT as R
import import_ipynb
import setPath
from os import listdir
from os.path import isfile, join
from Input.OpenDataPandaFramework13TeV import *
%jsroot on

Welcome to JupyROOT 6.24/02
importing Jupyter notebook from setPath.ipynb
importing Jupyter notebook from /storage/galaxy/jobs_directory/002/2793/working/jupyter/Input/OpenDataPandaFramework13TeV.ipynb
This library contains handy functions to ease the access and use of the 13TeV ATLAS OpenData release

getBkgCategories()
	 Dumps the name of the various background cataegories available 
	 as well as the number of samples contained in each category.
	 Returns a vector with the name of the categories

getSamplesInCategory(cat)
	 Dumps the name of the samples contained in a given category (cat)
	 Returns dictionary with keys being DSIDs and values physics process name from filename.

getMCCategory()
	 Returns dictionary with keys DSID and values MC category

initialize(indir)
	 Collects all the root files available in a certain directory (indir)



Setting luminosity to 10064 pb^-1

###############################
#### Background categories ####
###############################
Category    

## 1. Reading the dataset

Set the analaysis to run (*1largeRjet1lep*, *1lep1tau*, *3lep*, *exactly2lep*, *GamGam*, *2lep*, *4lep*)

Set the directory where you have downloaded the ATLAS OpenData samples you want to run over

In [2]:
opendatadir = "/storage/shared/data/fys5555/ATLAS_opendata/"
analysis = "4lep"

In [3]:
background = R.TChain("mini")
data = R.TChain("mini")

A list of all the background samples, category and their IDs can be found in **Infofile.txt**. The cross-section, efficiencies etc. needed for scaling are stored in the **Files_<---->**. We read these files and add all the samples to the TChain. We also (for later convenience) make a vector containing the dataset IDs. 

In [4]:
mcfiles = initialize(opendatadir+"/"+analysis+"/MC")
datafiles = initialize(opendatadir+"/"+analysis+"/Data")
allfiles = z = {**mcfiles, **datafiles}
Backgrounds = getBkgCategories()
#removeIndx = [2,3-1,4-2,6-3,7-4]
removeIndx = [3,4-1,5-2,6-3,7-4]
for indx in removeIndx: 
    Backgrounds.pop(indx)
print(Backgrounds)
Higgs = Backgrounds.pop(1); others = Backgrounds.pop(1); Backgrounds.insert(1,Higgs); Backgrounds.insert(0,others);  
print(Backgrounds)



###############################
#### Background categories ####
###############################
Category             N(samples)
-------------------------------
Diboson                       9
Higgs                         2
Other                         4
Wjets                        42
Wjetsincl                     6
Zjets                        42
singleTop                     6
topX                          3
['Diboson', 'Higgs', 'Other']
['Other', 'Diboson', 'Higgs']


In [5]:
MCcat = {}
for cat in allfiles:
    for dsid in allfiles[cat]["dsid"]:
        try:
            MCcat[int(dsid)] = cat
        except:
            continue

In [6]:
dataset_IDs = []
background.Reset()
for b in Backgrounds:
    i = 0
    if not b in mcfiles.keys(): continue
    for mc in mcfiles[b]["files"]:
        if not os.path.isfile(mc): continue
        try:
            dataset_IDs.append(int(mcfiles[b]["dsid"][i]))
            background.Add(mc)
        except:
            print("Could not get DSID for %s. Skipping"%mc)
        i += 1
nen = background.GetEntries()
print("Added %i entries for backgrounds"%(nen))

Added 922534 entries for backgrounds


In [7]:
data.Reset(); 
for d in datafiles["data"]["files"]:  
    if not os.path.isfile(d): continue
    data.Add(d)
nen = data.GetEntries()
print("Added %i entries for backgrounds"%(nen))

Added 832 entries for backgrounds


## 2. Making (a lot of) histograms

Now that we have read our dataset we want to start analyzing the data. To do so we need to put the data into histograms. For reasons that will become clear later in the analysis we must (for each variable) make one histogram per dataset ID. (We have 31 background samples, so if we want to study 10 variables we have to make 310 histograms!) For best dealing with all these histograms we can use dictionaries (Python) or maps (C++). 

In [45]:
hist_mllll = {}; hist_lep_pt = {}; hist_e = {}; hist_mll_1 = {}; hist_mll_2 = {}

In [46]:
min_val = 80 #80
max_val = 250 #250, 170
nr_bins = 34 #34, 24

min_val_m2 = 50 
max_val_m2 = 106 
nr_bins_m2 = 30 #30 

min_val_m3 = 10 
max_val_m3 = 140 
nr_bins_m3 = 30 #30 

min_val_p = 0 #80
max_val_p = 200 #250, 170
nr_bins_p = 5 #34, 50

min_val_e = 0 #80
max_val_e = 300 #250, 170
nr_bins_e = 10 #34, 50


In [47]:
for i in dataset_IDs: 
    hist_mllll[i] = R.TH1F() 
    hist_lep_pt[i] = R.TH1F()
    hist_e[i] = R.TH1F()
    hist_mll_1[i] = R.TH1F();
    hist_mll_2[i] = R.TH1F();


In [48]:
for i in dataset_IDs: 
    hist_mllll[i].SetNameTitle("hist_mllll", "Invariant mass"); 
    hist_lep_pt[i].SetNameTitle("hist_lep_pt", "Lepton pT"); 
    hist_e[i].SetNameTitle("hist_e", "Missing ET");
    hist_mll_1[i].SetNameTitle("hist_mll_1", "Di-lepton invariant mass (ee)");
    hist_mll_2[i].SetNameTitle("hist_mll_2", "Di-lepton invariant mass (mm)");
    
    hist_mll_1[i].SetBins(nr_bins_m2,min_val_m2,max_val_m2);  
    hist_mll_2[i].SetBins(nr_bins_m3,min_val_m3,max_val_m3); 
    hist_mllll[i].SetBins(nr_bins,min_val,max_val); 
    hist_lep_pt[i].SetBins(nr_bins_p, min_val_p, max_val_p);
    hist_e[i].SetBins(nr_bins_e, min_val_e, max_val_e); 

For data it is only necessary with one histogram for each variable: 

In [49]:
hist_mllll_d = R.TH1F(); 
hist_lep_pt_d = R.TH1F(); 
hist_e_d = R.TH1F();
hist_mll_1_d = R.TH1F();
hist_mll_2_d = R.TH1F();

In [50]:
hist_mllll_d.SetNameTitle("hist_mllll", "Invariant mass"); 
hist_lep_pt_d.SetNameTitle("hist_lep_pt", "Lepton pT"); 
hist_e_d.SetNameTitle("hist_e", "Missing ET");
hist_mll_1_d.SetNameTitle("hist_mll_1", "Di-lepton invariant mass (ee)"); 
hist_mll_2_d.SetNameTitle("hist_mll_2", "Di-lepton invariant mass (mm)"); 


hist_mllll_d.SetBins(nr_bins,min_val,max_val); 
hist_lep_pt_d.SetBins(nr_bins_p, min_val_p, max_val_p);
hist_e_d.SetBins(nr_bins_e, min_val_e, max_val_e);
hist_mll_1_d.SetBins(nr_bins_m2,min_val_m2,max_val_m2);
hist_mll_2_d.SetBins(nr_bins_m3,min_val_m3,max_val_m3); 

In [51]:
# Retrieve lumi from library
%store -r lumi

no stored variable or alias lumi


### 2.1 Fill the histograms 
We can now loop over all events in our dataset, implement desired cuts, and fill the histograms we created above. In this example we choose only events containing exactly to same flavour leptons with opposite charge (i.e. $e^+e^-$ or $\mu^+\mu^-$). 
Before starting the loop we extract the total number of entries (events) in the TChain. We also make [TLorentzVector](https://root.cern.ch/doc/master/classTLorentzVector.html)s, which are very practical for handling the kinematics of the leptons, e.g. calculating the invariant mass of the two leptons. 

In [52]:
for i in dataset_IDs: 
    hist_mllll[i].Reset(); 
    hist_lep_pt[i].Reset(); 
    hist_e[i].Reset();
    hist_mll_1[i].Reset(); 
    hist_mll_2[i].Reset(); 

In [53]:
hist_mllll_d.Reset(); 
hist_lep_pt_d.Reset(); 
hist_e_d.Reset(); 
hist_mll_1_d.Reset();
hist_mll_2_d.Reset();

In [54]:
l1 = R.TLorentzVector() 
l2 = R.TLorentzVector() 
l3 = R.TLorentzVector() 
l4 = R.TLorentzVector() 
quadraleptons = R.TLorentzVector() 
dilepton1 = R.TLorentzVector() 
dilepton2 = R.TLorentzVector() 

This is the cell where the analysis is performed. Note that the cell needs to be run twice:

1. with data = 0 to run over MC
2. with data = 1 to run over data

Note that the MC running takes ~5 minutes for 3lep analysis. Much(!!!) more time for e.g. 2lep analysis! Data running is relatively fast for 3lep. 

In [56]:
%%time
import time
from math import sin
isData = 0; 

if isData == 1: ds = data 
else: ds = background     

flavour_combo = [11*4, 13*4, 13*2 + 11*2]
i = 0  
z_mass = 91.18 #GeV
channels = []
for event in ds: 
    
    if i%100000 == 0 and i>0: 
        print("Total events %i/%i"%(i,ds.GetEntries()))
    i += 1 
    #if i == 10000: break
    
    ## Data quality cuts: 
    ## Event selection:
    if not (ds.trigE == 1 or ds.trigM == 1) : continue
    ## Cut #1: Require (exactly) 4 leptons
    if not ds.lep_n == 4: continue
    ## Cut #2: Require opposite charge
    if not (ds.lep_charge[0] + ds.lep_charge[1] + ds.lep_charge[2] + ds.lep_charge[3]) == 0 : continue
    ## Cut #3: Require same flavour (2 electrons or 2 muons)
    if not (ds.lep_type[0] + ds.lep_type[1] + ds.lep_type[2] + ds.lep_type[3]) in flavour_combo: continue
    
    
    ## Require "good leptons":
    if ds.lep_ptcone30[0] / ds.lep_pt[0] > 0.15: continue
    if ds.lep_etcone20[0] / ds.lep_pt[0] > 0.15: continue
    
    if ds.lep_ptcone30[1] / ds.lep_pt[1] > 0.3: continue
    if ds.lep_etcone20[1] / ds.lep_pt[1] > 0.3: continue
        
    if ds.lep_ptcone30[2] / ds.lep_pt[2] > 0.3: continue
    if ds.lep_etcone20[2] / ds.lep_pt[2] > 0.3: continue
    
    if ds.lep_ptcone30[3] / ds.lep_pt[3] > 0.3: continue
    if ds.lep_etcone20[3] / ds.lep_pt[3] > 0.3: continue
    
    
    if ds.lep_pt[0]/1000. < 25.: continue
    if ds.lep_pt[1]/1000. < 15.: continue
    if ds.lep_pt[2]/1000. < 10.: continue
    if ds.lep_pt[3]/1000. < 7.: continue
    
    # Electron(11) or muon(13):
    if ds.lep_type[0] == 11:
        if abs(ds.lep_eta[0]) > 2.47: continue
        if abs(ds.lep_eta[0]) > 1.37 and abs(ds.lep_eta[0]) < 1.52: continue
        if abs(ds.lep_trackd0pvunbiased[0] / ds.lep_tracksigd0pvunbiased[0]) > 5: continue
    else:
        if abs(ds.lep_eta[0]) > 2.5: continue
        if abs(ds.lep_trackd0pvunbiased[0] / ds.lep_tracksigd0pvunbiased[0]) > 3: continue 
        
    if ds.lep_type[1] == 11:
        if abs(ds.lep_eta[1]) > 2.47: continue
        if abs(ds.lep_eta[1]) > 1.37 and abs(ds.lep_eta[1]) < 1.52: continue
        if abs(ds.lep_trackd0pvunbiased[1] / ds.lep_tracksigd0pvunbiased[1]) > 5: continue
    else:
        if abs(ds.lep_eta[1]) > 2.5: continue
        if abs(ds.lep_trackd0pvunbiased[1] / ds.lep_tracksigd0pvunbiased[1]) > 3: continue
        
    if ds.lep_type[2] == 11:
        if abs(ds.lep_eta[2]) > 2.47: continue
        if abs(ds.lep_eta[2]) > 1.37 and abs(ds.lep_eta[2]) < 1.52: continue
        if abs(ds.lep_trackd0pvunbiased[2] / ds.lep_tracksigd0pvunbiased[2]) > 5: continue
    else:
        if abs(ds.lep_eta[2]) > 2.5: continue
        if abs(ds.lep_trackd0pvunbiased[2] / ds.lep_tracksigd0pvunbiased[2]) > 3: continue
        
    if ds.lep_type[3] == 11:
        if abs(ds.lep_eta[3]) > 2.47: continue
        if abs(ds.lep_eta[3]) > 1.37 and abs(ds.lep_eta[3]) < 1.52: continue
        if abs(ds.lep_trackd0pvunbiased[3] / ds.lep_tracksigd0pvunbiased[3]) > 5: continue
    else:
        if abs(ds.lep_eta[3]) > 2.5: continue
        if abs(ds.lep_trackd0pvunbiased[3] / ds.lep_tracksigd0pvunbiased[3]) > 3: continue

    
    
    ## Set Lorentz vectors: 
    l1.SetPtEtaPhiE(ds.lep_pt[0]/1000., ds.lep_eta[0], ds.lep_phi[0], ds.lep_E[0]/1000.);
    l2.SetPtEtaPhiE(ds.lep_pt[1]/1000., ds.lep_eta[1], ds.lep_phi[1], ds.lep_E[1]/1000.);
    l3.SetPtEtaPhiE(ds.lep_pt[2]/1000., ds.lep_eta[2], ds.lep_phi[2], ds.lep_E[2]/1000.);
    l4.SetPtEtaPhiE(ds.lep_pt[3]/1000., ds.lep_eta[3], ds.lep_phi[3], ds.lep_E[3]/1000.);
    
    
    if abs(ds.lep_z0[0] * sin(l1.Theta())) > 0.5: continue 
    
    if abs(ds.lep_z0[1] * sin(l2.Theta())) > 0.5: continue 
    
    if abs(ds.lep_z0[2] * sin(l3.Theta())) > 0.5: continue 
    
    if abs(ds.lep_z0[3] * sin(l4.Theta())) > 0.5: continue 
    
    ## Variables are stored in the TTree with unit MeV, so we need to divide by 1000 
    ## to get GeV, which is a more practical and commonly used unit. 
    
    quadraleptons = l1 + l2 + l3 + l4;
    
    if (ds.lep_type[0] + ds.lep_type[1] + ds.lep_type[2] + ds.lep_type[3]) in [44,52]:
        m_1 = abs((l1+l2).M()-z_mass)
        m_2 = abs((l1+l3).M()-z_mass)
        m_3 = abs((l1+l4).M()-z_mass)
        m_4 = abs((l2+l3).M()-z_mass)
        m_5 = abs((l2+l4).M()-z_mass)
        m_6 = abs((l3+l4).M()-z_mass)
        if ds.lep_charge[0] == ds.lep_charge[1]:
            if (m_2 < m_3):
                dilepton_1 = l1 + l3
                dilepton_2 = l2 + l4
            else:
                dilepton_1 = l1 + l4
                dilepton_2 = l2 + l3
                
        elif ds.lep_charge[0] == ds.lep_charge[2]:
            if (m_1 < m_3):
                dilepton_1 = l1 + l2
                dilepton_2 = l3 + l4
            else:
                dilepton_1 = l1 + l4
                dilepton_2 = l2 + l3
        elif ds.lep_charge[0] == ds.lep_charge[3]:
            if (m_1 < m_2):
                dilepton_1 = l1 + l2
                dilepton_2 = l3 + l4
            else:
                dilepton_1 = l1 + l3
                dilepton_2 = l2 + l4
       
    else:
        if ds.lep_type[0] == ds.lep_type[1]:
            dilepton_1 = l1 + l2;
            dilepton_2 = l3 + l4;
        elif ds.lep_type[0] == ds.lep_type[2]: 
            dilepton_1 = l1 + l3;
            dilepton_2 = l2 + l4;
        else:
            dilepton_1 = l1 + l4;
            dilepton_2 = l2 + l3;

    if abs(dilepton_1.M() - z_mass) < abs(dilepton_2.M() - z_mass) :
        z_1 = dilepton_1
        z_2 = dilepton_2
    else:
        z_1 = dilepton_2
        z_2 = dilepton_1
        
    if isData == 1:

        hist_mllll_d.Fill(quadraleptons.M());
        
        hist_lep_pt_d.Fill(l1.Pt());
        hist_lep_pt_d.Fill(l2.Pt());
        hist_lep_pt_d.Fill(l3.Pt());
        hist_lep_pt_d.Fill(l4.Pt());
        
        hist_e_d.Fill(l1.E());
        hist_e_d.Fill(l2.E());
        hist_e_d.Fill(l3.E());
        hist_e_d.Fill(l4.E());
        
        hist_mll_1_d.Fill(z_1.M());
        hist_mll_2_d.Fill(z_2.M());
 
        
    else: 
        #(ds.scaleFactor_LepTRIGGER)
        W = ((ds.mcWeight)*(ds.scaleFactor_PILEUP)*
             (ds.scaleFactor_ELE)*(ds.scaleFactor_MUON)*
             (ds.scaleFactor_LepTRIGGER))*((ds.XSection*lumi)/ds.SumWeights)
        
        hist_mllll[ds.channelNumber].Fill(quadraleptons.M(), W);
        
        hist_lep_pt[ds.channelNumber].Fill(l1.Pt(), W);
        hist_lep_pt[ds.channelNumber].Fill(l2.Pt(), W);
        hist_lep_pt[ds.channelNumber].Fill(l3.Pt(), W); 
        hist_lep_pt[ds.channelNumber].Fill(l4.Pt(), W); 
        
        hist_e[ds.channelNumber].Fill(l1.E(), W);
        hist_e[ds.channelNumber].Fill(l2.E(), W);
        hist_e[ds.channelNumber].Fill(l3.E(), W); 
        hist_e[ds.channelNumber].Fill(l4.E(), W);
        
        hist_mll_1[ds.channelNumber].Fill(z_1.M(),W);
        hist_mll_2[ds.channelNumber].Fill(z_2.M(),W);
print("Done!")
if isData == 0:
    print("Remebered to run over data? No? Set data = 1 at the top and run again")
else:
    print("Remebered to run over MC? No? Set data = 0 at the top and run again")

Total events 100000/922534
Total events 200000/922534
Total events 300000/922534
Total events 400000/922534
Total events 500000/922534
Total events 600000/922534
Total events 700000/922534
Total events 800000/922534
Total events 900000/922534
Done!
Remebered to run over data? No? Set data = 1 at the top and run again
CPU times: user 3min 54s, sys: 326 ms, total: 3min 54s
Wall time: 3min 55s


## 3. Scale and classify the histograms (MC only) 

Before we are ready to make plots we need to do some further processing of the histograms we made above. The information necessary for doing the two steps below is found in the file **Infofile.txt**.   
1. We need to **scale** the histograms to the right cross section and luminosity. Why? When making the MC samples a certain number of events is simulated, which will usually not correspond to the number of events in our data. The expected number of events from a certain kind of process is given by $N=\sigma L$, where $\sigma$ is the cross section and $L$ is the integrated luminosity. Therefore we need to scale each histogram by a scale factor <br> <br>
$$sf = \frac{N}{N_{MC}} = \frac{ \sigma L }{N_{MC}},$$ <br>  where $N_{MC}$ is the number of generated MC events.  <br> <br>
2. We also need to **classify** the background processes into different categories. This is necessary when we eventually want to make the characteristic colorful background plots you might have seen before.  

### 3.1 Make new histograms 
Maybe a bit depressingly we have to make a set of new histograms, this time corresponding to the different background categories, instead of the dataset IDs. Notice that these new histograms are made in a very similar way as above, i.e. with the same range and binning. 

In [57]:
H_mllll = {}; H_lep_pt = {}; H_e = {}; H_mll_1 = {}; H_mll_2 = {};

In [58]:
L = 10.06
include_higgs = True

In [59]:
for i in Backgrounds: 
    if i == "Higgs" and not include_higgs: continue
    H_mllll[i] = R.TH1F() 
    H_lep_pt[i] = R.TH1F() 
    H_e[i] = R.TH1F()
    H_mll_1[i] = R.TH1F()
    H_mll_2[i] = R.TH1F()
    

In [60]:
for i in Backgrounds: 
    if i == "Higgs" and not include_higgs: continue
    H_mllll[i].SetNameTitle("hist_mll", "Invariant mass"); 
    H_lep_pt[i].SetNameTitle("hist_lep_pt", "Lepton pT"); 
    H_e[i].SetNameTitle("hist_e", "Missing ET");
    H_mll_1[i].SetNameTitle("hist_mll_1", "Di-lepton invariant mass (ee)");
    H_mll_2[i].SetNameTitle("hist_mll_2", "Di-lepton invariant mass (mm)");  
    H_mllll[i].SetBins(nr_bins,min_val,max_val); 
    H_lep_pt[i].SetBins(nr_bins_p, min_val_p, max_val_p);
    H_e[i].SetBins(nr_bins_e, min_val_e, max_val_e);
    H_mll_1[i].SetBins(nr_bins_m2,min_val_m2,max_val_m2);
    H_mll_2[i].SetBins(nr_bins_m3,min_val_m3,max_val_m3);

### 3.2 Scale and add histograms 
Now we read our info file, scale all (old) histograms, and then add them to the new histograms we just defined.  

In [61]:
for i in Backgrounds:
    if i == "Higgs" and not include_higgs: continue
    H_mllll[i].Reset(); 
    H_lep_pt[i].Reset(); 
    H_e[i].Reset();
    H_mll_1[i].Reset();
    H_mll_2[i].Reset();

In [62]:
for dsid in hist_mllll.keys(): 
    
    Type = MCcat[dsid]
    if Type == "Higgs" and not include_higgs: continue
    H_mllll[Type].Add(hist_mllll[dsid]); 
    H_lep_pt[Type].Add(hist_lep_pt[dsid]); 
    H_e[Type].Add(hist_e[dsid]);
    H_mll_1[Type].Add(hist_mll_1[dsid]);
    H_mll_2[Type].Add(hist_mll_2[dsid]);

H_mllll["Diboson"].Scale(1.3)
H_lep_pt["Diboson"].Scale(1.3); 
H_e["Diboson"].Scale(1.3);
H_mll_1["Diboson"].Scale(1.3);
H_mll_2["Diboson"].Scale(1.3);

infofile.close()

### 3.3 Color the histograms 
Make yet another map, this time containing the colors you want the backgrounds to have, and then set the colors of your histograms. Note that colors are defined by integers in ROOT. If you are not happy with the colors chosen below you can have look at the [TColor](https://root.cern.ch/doc/master/classTColor.html) class reference for more options. 

In [63]:
colors = {}

In [64]:
colors["Diboson"] = R.kCyan; 
colors["Zjets"] = R.kYellow; 
colors["ttbar"] = R.kBlue+3;
colors["singleTop"] = R.kBlue-7; 
colors["Wjets"] = R.kBlue+3; 
colors["topX"] = R.kOrange+1; 
colors["Higgs"] = R.kRed; 
colors["Wjetsincl"] = R.kBlue-10;
colors["Zjetsincl"] = R.kBlue+3;
colors["Other"] = R.kBlue+2;

In [65]:
for h in Backgrounds:
    if h == "Higgs" and not include_higgs: continue
    H_mllll[h].SetFillColor(colors[h]); 
    H_e[h].SetFillColor(colors[h]);
    H_lep_pt[h].SetFillColor(colors[h]);
    
    H_mllll[h].SetLineColor(colors[h]); 
    H_e[h].SetLineColor(colors[h]);
    H_lep_pt[h].SetLineColor(colors[h]);
    
    H_mll_1[h].SetFillColor(colors[h]);
    H_mll_2[h].SetFillColor(colors[h]);
    
    H_mll_1[h].SetLineColor(colors[h]);
    H_mll_2[h].SetLineColor(colors[h]);

## 4. Stack and plot the histograms

Finally we have arrived to the part where we can plot the results of all the work done above. For each variable we need to stack the backgrounds on top of each other, which is done by using the [THStack](https://root.cern.ch/doc/master/classTHStack.html) class. In the example below we do this for two variables; invariant mass and missing $E_T$.   

In [66]:
stack_mllll = R.THStack("Invariant mass", "");
stack_e = R.THStack("Missing ET", ""); 
stack_lep_pt = R.THStack("Lepton pT", "");
stack_mll_1 =  R.THStack("Di-lepton invariant mass (ee)", "");
stack_mll_2 =  R.THStack("Di-lepton invariant mass (mm)", ""); 

In [67]:
for h in Backgrounds:
    if h == "Higgs" and not include_higgs : continue
    stack_mllll.RecursiveRemove(H_mllll[h]); ## Remove previously stacked histograms  
    stack_e.RecursiveRemove(H_e[h]);
    stack_lep_pt.RecursiveRemove(H_lep_pt[h]);
    stack_mll_1.RecursiveRemove(H_mll_1[h]);
    stack_mll_2.RecursiveRemove(H_mll_2[h]);
    stack_mllll.Add(H_mllll[h]);
    stack_e.Add(H_e[h]);
    stack_lep_pt.Add(H_lep_pt[h]); 
    stack_mll_1.Add(H_mll_1[h]); 
    stack_mll_2.Add(H_mll_2[h]);

Now we make a legend (see [TLegend](https://root.cern.ch/doc/master/classTLegend.html)), and add  the different backgrounds. Next we make a canvas (see [TCanvas](https://root.cern.ch/doc/master/classTCanvas.html)), which is allways necessary when we want to make a plot. Then you draw the stack and the legend, and display them by drawing the canvas. We can also specify axis labels and a bunch of other stuff. 

In [68]:
R.gStyle.SetLegendBorderSize(0); ## Remove (default) border around legend 
leg = R.TLegend(0.65, 0.60, 0.9, 0.85); 

In [69]:
leg.Clear();
for i in Backgrounds:
    if i == "Higgs" and not include_higgs: continue
    leg.AddEntry(H_mllll[i], i, "f")  ## Add your histograms to the legend
leg.AddEntry(hist_mllll_d, "Data", "lep") 

In [70]:
C = R.TCanvas("c", "c", 600, 600)

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c


In [71]:
#R.gPad.SetLogy() ## Set logarithmic y-axis

In [72]:
hist_mllll_d.SetLineColor(R.kBlack); 
hist_mllll_d.SetMarkerStyle(R.kFullCircle); 
hist_mllll_d.SetMarkerColor(R.kBlack); 

In [73]:
stack_mllll.Draw("hist");
stack_mllll.Draw("hist"); 
stack_mllll.SetMaximum(40); 
stack_mllll.SetMinimum(0); 
stack_mllll.GetYaxis().SetTitle("# Events/bin");
stack_mllll.GetYaxis().SetTitleOffset(1.3); 
stack_mllll.GetXaxis().SetTitle("m_{llll} (GeV)");
stack_mllll.GetXaxis().SetTitleOffset(1.3);
hist_mllll_d.Draw("same E"); 
leg.Draw();
C.SaveAs("m_llll_250.pdf");
C.Draw();


Info in <TCanvas::Print>: pdf file m_llll_250.pdf has been created


In [37]:
#R.gPad.SetLogy() ## Set logarithmic y-axis
hist_e_d.SetLineColor(R.kBlack); 
hist_e_d.SetMarkerStyle(R.kFullCircle); 
hist_e_d.SetMarkerColor(R.kBlack); 

In [38]:
stack_e.Draw("hist"); 
stack_e.SetMaximum(900); 
stack_e.SetMinimum(0); 
stack_e.GetYaxis().SetTitle("# Events/bin");
stack_e.GetYaxis().SetTitleOffset(1.3); 
stack_e.GetXaxis().SetTitle("E_{all} (GeV)");
stack_e.GetXaxis().SetTitleOffset(1.3);
hist_e_d.Draw("same e"); 
leg.Draw();
C.SaveAs("E_all.pdf");
C.Draw(); 

Info in <TCanvas::Print>: pdf file E_all.pdf has been created


In [39]:
hist_lep_pt_d.SetLineColor(R.kBlack); 
hist_lep_pt_d.SetMarkerStyle(R.kFullCircle); 
hist_lep_pt_d.SetMarkerColor(R.kBlack); 

In [40]:
stack_lep_pt.Draw("hist"); 
stack_lep_pt.SetMaximum(1900); 
stack_lep_pt.GetYaxis().SetTitle("# Events/bin");
stack_lep_pt.GetYaxis().SetTitleOffset(1.3); 
stack_lep_pt.GetXaxis().SetTitle("p_{T} (GeV)");
stack_lep_pt.GetXaxis().SetTitleOffset(1.3);
hist_lep_pt_d.Draw("same e"); 
leg.Draw();
C.SaveAs("P_t.pdf");
C.Draw(); 

Info in <TCanvas::Print>: pdf file P_t.pdf has been created


In [41]:
hist_mll_1_d.SetLineColor(R.kBlack); 
hist_mll_1_d.SetMarkerStyle(R.kFullCircle); 
hist_mll_1_d.SetMarkerColor(R.kBlack); 

In [42]:
stack_mll_1.Draw("hist"); 
stack_mll_1.Draw("hist"); 
stack_mll_1.SetMaximum(140); 
#stack_mll_1.SetMinimum(1); 
stack_mll_1.GetYaxis().SetTitle("# Events/bin");
stack_mll_1.GetYaxis().SetTitleOffset(1.3); 
stack_mll_1.GetXaxis().SetTitle("m_{ll,1} (GeV)");
stack_mll_1.GetXaxis().SetTitleOffset(1.3);
stack_mll_1.Draw("same E"); 
hist_mll_1_d.Draw("same e");
leg.Draw();
C.SaveAs("m_ll1.pdf");
C.Draw();

Info in <TCanvas::Print>: pdf file m_ll1.pdf has been created


In [43]:
hist_mll_2_d.SetLineColor(R.kBlack); 
hist_mll_2_d.SetMarkerStyle(R.kFullCircle); 
hist_mll_2_d.SetMarkerColor(R.kBlack); 

In [44]:
stack_mll_2.Draw("hist"); 
stack_mll_2.SetMaximum(100); 
stack_mll_2.SetMinimum(0); 
stack_mll_2.GetYaxis().SetTitle("# Events/bin");
stack_mll_2.GetYaxis().SetTitleOffset(1.3); 
stack_mll_2.GetXaxis().SetTitle("m_{ll,2} (GeV)");
stack_mll_2.GetXaxis().SetTitleOffset(1.3);
hist_mll_2_d.Draw("same E");
leg.Draw();
C.SaveAs("m_ll2.pdf");
C.Draw();

Info in <TCanvas::Print>: pdf file m_ll2.pdf has been created
